# Scraping from EDINET:
1. Before we download all reports, we need to get a sense of the universe. The full universe can be found in
https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.bean=ee.bean.W1E62071.EEW1E62071Bean&uji.verb=W1E62071InitDisplay&TID=W1E62071&PID=W0EZ0001&SESSIONKEY=&lgKbn=2&dflg=0&iflg=0


# Next step: systematic downloads
1. Want to download all htmls listed on that page
2. Also want to save the associated details of the page itself

In [1]:
xlnk = 'https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?'+\
      'uji.verb=W1E63021CXP002002DSPSch&uji.bean=ee.bean.parent.'+\
      'EECommonSearchBean&PID=W1E63021&TID=W1E63021&SESSIONKEY='+\
      '1537198278881&lgKbn=2&pkbn=0&skbn=1&dskb=&askb=&dflg=0&iflg=0'+\
      '&preId=1&sec=&scc={code}&shb=&snm=&spf1=1&spf2=1&iec=&icc=&inm=&spf3=1&'+\
      'fdc=&fnm=&spf4=1&spf5=2&otd=120&cal=1&era=H&yer=&mon=&psr=1&pfs=5&'+\
      'row=100&idx=0&str=&kbn=1&flg=&syoruiKanriNo='

In [125]:
import time
import numpy as np
import pandas as pd
import unicodedata


#from PIL import Image
from io import StringIO


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

%matplotlib inline
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML


options = Options()
# Chromeのパス（Stableチャネルで--headlessが使えるようになったら不要なはず）
options.binary_location = ''
# ヘッドレスモードを有効にする（次の行をコメントアウトすると画面が表示される）。
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--start-maximized')
options.add_argument('--disable-gpu')
options.add_argument('--allow-insecure-localhost')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument('--disable-web-security')

DesiredCapabilities.CHROME.update({'acceptInsecureCerts': True})

# ChromeのWebDriverオブジェクトを作成する。
fn = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe'
driver = webdriver.Chrome(fn, chrome_options=options, 
                          desired_capabilities=DesiredCapabilities.CHROME)

# Googleのトップ画面を開く。
#driver.get(lnk)
%matplotlib inline
import matplotlib.pyplot as plt

C:\Users\Pawlovicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: use options instead of chrome_options


In [77]:
driver.quit()

In [120]:
def click_link_in_left_menu(driver, down_cat=u'大株主の状況'):
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('viewFrame')
    driver.switch_to.frame('menuFrame2')
    #driver.switch_to_frame('mainFrame')
    lnk2 = driver.find_elements_by_partial_link_text(down_cat)[0]
    lnk2.click()
    return driver

def focus_on_main_frame(driver):
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('viewFrame')
    driver.switch_to.frame('mainFrame')
    return driver

In [121]:
def extract_holder_info_html(driver):
    down_cat = u'大株主の状況'
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('viewFrame')
    driver.switch_to.frame('menuFrame2')
    #driver.switch_to_frame('mainFrame')
    lnk2 = driver.find_elements_by_partial_link_text(down_cat)[0]
    lnk2.click()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('viewFrame')
    driver.switch_to.frame('mainFrame')
    return driver.page_source

def wareki_to_date(date_str):
    date_str = date_str.replace('元', '0')
    if '平成' in date_str:
        year = int(re.findall(r"(\d+)年", date_str)[0]) + 1988
        date_str = str(year) + '年' + date_str.split('年')[-1]
    elif '令和' in date_str:
        year = int(re.findall(r"(\d+)年", date_str)[0]) + 2019
        date_str = str(year) + '年' + date_str.split('年')[-1]
    date = pd.datetime.strptime(date_str, '%Y年%m月%d日')
    return date

def extract_date_from_front_page(driver):
    dt_tg_nm = 'jpcrp_cor:FilingDateCoverPage'
    tagel = driver.find_elements_by_name(dt_tg_nm)
    date_str = unicodedata.normalize("NFKC",tagel[0].text)
    date_str = ''.join(date_str.split())
    return wareki_to_date(date_str)


def save_file(filename, page_source):
    fn = open(filename,"w", encoding="utf-8")
    fn.write(page_source)
    fn.close()

In [1]:
def prepare_record_dataframe(rdf):
    def wareki_asci_to_year(date_str):
        year = int(re.findall(r"(\d+)\.", date_str)[0])
        if 'H' in date_str:
            year = year + 1988  
        elif 'R' in date_str:
            year = year + 2018
        date_str = str(year) + '.' + '.'.join(date_str.split('.')[1:])
        return date_str
    rdf = pd.DataFrame(rdf.iloc[1:, :].values, columns=rdf.iloc[0])
    rdf.loc[:, 'data_date'] = pd.to_datetime(rdf['提出日時'].apply(wareki_asci_to_year), errors='coerce')
    return rdf

In [2]:
import re
fns = []
def extract_holders_by_edinetcode(driver, edinetcode, ticker, xlnk):
    url = xlnk.format(**{'edinetcode': edinetcode})
    test2 = driver.get(url)
    #print(driver.page_source)
    print(url)
    rdf = pd.read_html(driver.page_source)[-1]
    yulinks = driver.find_elements_by_partial_link_text(u'有証券報告書')
    yulinks = [x for x in yulinks if re.match('^有価証券報告書', x.text)]
    for i, yulink in enumerate(yulinks):
        out_text = edinetcode + ' - ' + yulink.text
        print(out_text)
        yulink.click()
        focus_on_main_frame(driver)
        try:
            date = extract_date_from_front_page(driver)
            click_link_in_left_menu(driver, u'大株主の状況')
        except IndexError:
            print(out_text +'... failed to retrieve')
        focus_on_main_frame(driver)
        fn = 'data/' + 'topix/' + ticker + u'_' +\
             date.strftime('%Y%m%d')+'.html'
        save_file(fn, driver.page_source)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        fns.append(fn)
    rdf.loc[:, 'filename'] = fns
    return rdf
    #driver.close()

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
from YuhoScraper import YuhoScraper
from YuhoScraper import download_all_yuhos_by_edinetcode
import pandas as pd
import os

In [2]:
from IndexMaster import IndexMaster
from EDINETUniverse import EDINETUniverse

im = IndexMaster().parse_files().rename_columns().add_ts_features()\
                  .add_event_flags().get_topix_ffw_changes()
euni = EDINETUniverse().get_edinet_list().get_topix_universe()\
                       .add_edinet_code()\
                       .add_features()
tdf = im.loc[im['date'] == '2021-01-29']
tdf = tdf.merge(euni.edf.loc[:, ['ticker', 'ＥＤＩＮＥＴコード']],
          on='ticker', how='left')
tdf = tdf.loc[~tdf['ＥＤＩＮＥＴコード'].isnull()]

In [ ]:
fo

In [39]:
fdir = 'C:/Users/Pawlovicky/Documents/fast_data/privat/trading/TOPIX_FFW/topix_ffw_estimation/output/kaishajokyo'
fns = os.listdir(fdir)
fdf = pd.DataFrame(fns)
cecodes = fdf[0].str.split('_').str.get(0).unique()

In [40]:
cecodes

array(['1883', '2681', '4430', ..., 'E35488', 'E35810', 'E35931'],
      dtype=object)

In [41]:
mecodes = [x for x in ecodes if not x in cecodes]

In [42]:
len(mecodes)

40

In [43]:
for i, ecode in enumerate(ecodes):
    if ecode == 'E01488':
        print(i)

1902


In [46]:
failedlist = []
mdf = []
for x in mecodes:
    try:
        idf = download_all_yuhos_by_edinetcode(x)
        mdf.append(idf)
    except:
        failedlist.append(x)
mdf = pd.concat(mdf)

In [47]:
failedlist

['E04236', 'E02126', 'E04239', 'E02529', 'E02213', 'E02558', 'E01321']

In [44]:
rdf = list(map(download_all_yuhos_by_edinetcode, mecodes))

TimeoutException: Message: 


In [38]:
with Pool(10) as p:
    rdf = p.map(download_all_yuhos_by_edinetcode, mecodes)

TimeoutException: Message: 


In [8]:
%%time
ecodes = tdf['ＥＤＩＮＥＴコード'].tolist()
from multiprocessing import Pool
import itertools as it
with Pool(10) as p:
    mdf = p.map(download_all_yuhos_by_edinetcode, ecodes)
#with Pool(2) as p:
#    mdf = p.map(download_all_yuhos_by_edinetcode, ecodes)

WebDriverException: Message: unknown error: DevToolsActivePort file doesn't exist


In [7]:
pd.concat(mdf)

,filingdatetime,name,edinetcode,issuername,,status
0,2021-06-21 15:00:00,有価証券報告書－第15期(令和2年4月1日－令和3年3月31日),E31755,株式会社かんぽ生命保険,,SUCCESS
0,2020-06-18 15:00:00,有価証券報告書－第14期(平成31年4月1日－令和2年3月31日),E31755,株式会社かんぽ生命保険,,SUCCESS
0,2019-06-20 15:00:00,有価証券報告書－第13期(平成30年4月1日－平成31年3月31日),E31755,株式会社かんぽ生命保険,,SUCCESS
0,2018-06-21 15:06:00,有価証券報告書－第12期(平成29年4月1日－平成30年3月31日),E31755,株式会社かんぽ生命保険,,SUCCESS
0,2017-06-23 15:00:00,有価証券報告書－第11期(平成28年4月1日－平成29年3月31日),E31755,株式会社かんぽ生命保険,,SUCCESS
0,2021-06-29 15:00:00,有価証券報告書－第48期(令和2年4月1日－令和3年3月31日),E03943,株式会社レオパレス２１,,SUCCESS
0,2020-07-22 15:00:00,有価証券報告書－第47期(平成31年4月1日－令和2年3月31日),E03943,株式会社レオパレス２１,,SUCCESS
0,2019-06-28 14:30:00,有価証券報告書－第46期(平成30年4月1日－平成31年3月31日),E03943,株式会社レオパレス２１,,SUCCESS
0,2018-06-28 16:59:00,有価証券報告書－第45期(平成29年4月1日－平成30年3月31日),E03943,株式会社レオパレス２１,,SUCCESS
0,2017-06-29 17:01:00,有価証券報告書－第44期(平成28年4月1日－平成29年3月31日),E03943,株式会社レオパレス２１,,SUCCESS


In [2]:
%%time
from EDINETUniverse import EDINETUniverse
euni = EDINETUniverse().get_edinet_list().get_topix_universe()\
                       .add_edinet_code()\
                       .add_features()
uni = euni.df
mdf = []
for i, row in uni.reset_index(drop=True).iterrows():
    print(row['ticker'])
    if i > 3:
        break
    xlnk = rawurl
    idf = download_all_yuhos_by_edinetcode(row['ＥＤＩＮＥＴコード'],
                                     row['ticker'])
    mdf.append(idf)
mdf = pd.concat(mdf)

7203


NameError: name 'rawurl' is not defined

In [5]:
from IndexMaster import IndexMaster
from EDINETUniverse import EDINETUniverse

im = IndexMaster().parse_files().rename_columns().add_ts_features()\
                  .add_event_flags().get_topix_ffw_changes()
euni = EDINETUniverse().get_edinet_list().get_topix_universe()\
                       .add_edinet_code()\
                       .add_features()

In [ ]:
ecodes = tdf['ＥＤＩＮＥＴコード'].tolist()[:4]
from multiprocessing import Pool
import itertools as it
with Pool(2) as p:
    mdf = p.map(download_all_yuhos_by_edinetcode, ecodes)
#with Pool(2) as p:
#    mdf = p.map(download_all_yuhos_by_edinetcode, ecodes)

In [17]:
res = map(lambda x: x*x, [1, 2, 3, 4])

In [18]:
res

In [19]:
list(mdf)

[       filingdatetime                                name edinetcode  \
 0 2021-06-21 15:00:00    有価証券報告書－第15期(令和2年4月1日－令和3年3月31日)     E31755   
 0 2020-06-18 15:00:00   有価証券報告書－第14期(平成31年4月1日－令和2年3月31日)     E31755   
 0 2019-06-20 15:00:00  有価証券報告書－第13期(平成30年4月1日－平成31年3月31日)     E31755   
 0 2018-06-21 15:06:00  有価証券報告書－第12期(平成29年4月1日－平成30年3月31日)     E31755   
 0 2017-06-23 15:00:00  有価証券報告書－第11期(平成28年4月1日－平成29年3月31日)     E31755   
 
       issuername      status  
 0  株式会社かんぽ生命保険       SUCCESS  
 0  株式会社かんぽ生命保険       SUCCESS  
 0  株式会社かんぽ生命保険       SUCCESS  
 0  株式会社かんぽ生命保険       SUCCESS  
 0  株式会社かんぽ生命保険       SUCCESS  ,
        filingdatetime                                name edinetcode  \
 0 2021-06-29 15:00:00    有価証券報告書－第48期(令和2年4月1日－令和3年3月31日)     E03943   
 0 2020-07-22 15:00:00   有価証券報告書－第47期(平成31年4月1日－令和2年3月31日)     E03943   
 0 2019-06-28 14:30:00  有価証券報告書－第46期(平成30年4月1日－平成31年3月31日)     E03943   
 0 2018-06-28 16:59:00  有価証券報告書－第45期(平成29年4月1日－平成30年3月31日)     E03943   
 0

In [12]:
mdf

In [12]:
%%time
mdf = []
for i, row in tdf.iterrows():
    print(row['ticker'])
    print(row['ＥＤＩＮＥＴコード'])
    idf = download_all_yuhos_by_edinetcode(row['ＥＤＩＮＥＴコード'],
                                     row['ticker'])
    mdf.append(idf)
mdf = pd.concat(mdf)

7181
E31755
Wall time: 13.5 s


In [295]:
tdf.head()

,date,ticker,name,changeinno.ofshares,no.ofshares,ffw,prev_ffw,dffw,abs_dffw
9225,2021-01-29,8184,"SHIMACHU CO.,LTD.",-1.917410e+07,10652276.0,0.25,0.7000,-0.4500,0.4500
8154,2021-01-29,4321,"Kenedix, Inc.",-8.823248e+07,44116240.0,0.20,0.6000,-0.4000,0.4000
9567,2021-01-29,9671,"YOMIURI LAND.CO.,LTD.",-2.505661e+06,835220.2,0.10,0.4000,-0.3000,0.3000
8879,2021-01-29,7181,"JAPAN POST INSURANCE Co.,Ltd.",1.406500e+08,196910000.0,0.35,0.1000,0.2500,0.2500
9382,2021-01-29,8848,LEOPALACE21 CORPORATION,-7.638543e+07,131755806.0,0.40,0.6319,-0.2319,0.2319


In [292]:
im.loc[im['name'].str.contains('SHIMA')]

,date,ticker,name,changeinno.ofshares,no.ofshares,ffw,prev_ffw,dffw,abs_dffw
9225,2021-01-29,8184,"SHIMACHU CO.,LTD.",-19174096.80,10652276.00,0.2500,0.7000,-0.4500,0.4500
8872,2021-01-29,7150,"THE SHIMANE BANK,LTD.",1288489.60,5470400.00,0.6500,0.4969,0.1531,0.1531
9328,2021-01-29,8562,"THE FUKUSHIMA BANK,LTD.",3250800.00,18200000.00,0.6500,0.5339,0.1161,0.1161
4916,2020-12-30,6464,"TSUBAKI NAKASHIMA CO.,LTD.",0.00,30797250.00,0.7500,0.8000,-0.0500,0.0500
8645,2021-01-29,6420,FUKUSHIMA GALILEI CO.LTD.,1103308.00,11033080.00,0.5000,0.4500,0.0500,0.0500
7494,2021-01-29,1518,"MITSUI MATSUSHIMA HOLDINGS CO.,LTD.",653220.00,9145080.00,0.7000,0.6500,0.0500,0.0500
8564,2021-01-29,6222,"SHIMA SEIKI MFG.,LTD.",-1790000.00,23270000.00,0.6500,0.7000,-0.0500,0.0500
5791,2020-12-30,9535,"HIROSHIMA GAS CO.,LTD.",0.00,33147656.70,0.4866,0.4875,-0.0009,0.0009
9543,2021-01-29,9535,"HIROSHIMA GAS CO.,LTD.",61308.86,33208965.56,0.4875,0.4866,0.0009,0.0009
8991,2021-01-29,7545,"NISHIMATSUYA CHAIN Co.,Ltd.",0.00,34794428.00,0.5000,0.5000,0.0000,0.0000


In [279]:
mdf.query('status == "FAILED"')

,filingdatetime,name,edinetcode,issuername,,status


In [280]:
mdf

,filingdatetime,name,edinetcode,issuername,,status
0,2021-06-24 15:00:00,有価証券報告書－第117期(令和2年4月1日－令和3年3月31日),E02144,トヨタ自動車株式会社,,SUCCESS
0,2020-06-24 15:00:00,有価証券報告書－第116期(平成31年4月1日－令和2年3月31日),E02144,トヨタ自動車株式会社,,SUCCESS
0,2019-06-21 15:00:00,有価証券報告書－第115期(平成30年4月1日－平成31年3月31日),E02144,トヨタ自動車株式会社,,SUCCESS
0,2018-06-25 15:00:00,有価証券報告書－第114期(平成29年4月1日－平成30年3月31日),E02144,トヨタ自動車株式会社,,SUCCESS
0,2017-06-23 16:04:00,有価証券報告書－第113期(平成28年4月1日－平成29年3月31日),E02144,トヨタ自動車株式会社,,SUCCESS
0,2021-06-22 15:15:00,有価証券報告書－第104期(令和2年4月1日－令和3年3月31日),E01777,ソニーグループ株式会社,,SUCCESS
0,2020-06-26 16:16:00,有価証券報告書－第103期(平成31年4月1日－令和2年3月31日),E01777,ソニーグループ株式会社,,SUCCESS
0,2019-06-18 15:01:00,有価証券報告書－第102期(平成30年4月1日－平成31年3月31日),E01777,ソニーグループ株式会社,,SUCCESS
0,2018-06-19 15:45:00,有価証券報告書－第101期(平成29年4月1日－平成30年3月31日),E01777,ソニーグループ株式会社,,SUCCESS
0,2017-06-15 15:00:00,有価証券報告書－第100期(平成28年4月1日－平成29年3月31日),E01777,ソニーグループ株式会社,,SUCCESS


In [251]:
from YuhoScraper import YuhoScraper
import os
with YuhoScraper() as ys:
    ecode = 'E02144'
    code = '7203'
    ys.get_search_page_from_edinetcode(ecode)
    ylinks = ys.get_yuho_links()
    linktexts = [ylink.text for ylink in ylinks]
    mdf = []
    for linktext in linktexts:
        print(linktext)
        try:
            ys.close_all_but_first_window()
            ylink = ys.get_yuho_link_by_text(linktext)
            idf = ys.get_metadata_for_ylink_as_dataframe(ylink)
            ys.click_yuho_link(ylink)
            ys.click_holdings_section()
            fn = ['output', 'kaishajokyo',
                  code + '_' + str(idf['filingdatetime'].dt.date.iloc[0])\
                                .replace('-', '')+'.html']
            fn = os.path.join(*fn)
            ys.save_content(fn)
            ys.close_all_but_first_window()
            idf.loc[:, 'status'] = 'SUCCESS'
        except:
            idf.loc[:, 'status'] = 'FAILED'
        mdf.append(idf)
mdf = pd.concat(mdf)

有価証券報告書－第117期(令和2年4月1日－令和3年3月31日)
有価証券報告書－第116期(平成31年4月1日－令和2年3月31日)
有価証券報告書－第115期(平成30年4月1日－平成31年3月31日)
有価証券報告書－第114期(平成29年4月1日－平成30年3月31日)
有価証券報告書－第113期(平成28年4月1日－平成29年3月31日)


In [242]:
idf['filingdatetime'].dt.date.iloc[0]

'2021-06-24'

In [234]:
mdf

[]

In [231]:
mdf

,filingdatetime,name,edinetcode,issuername,,status
0,2021-06-24 15:00:00,有価証券報告書－第117期(令和2年4月1日－令和3年3月31日),E02144,トヨタ自動車株式会社,,FAILED
0,2020-06-24 15:00:00,有価証券報告書－第116期(平成31年4月1日－令和2年3月31日),E02144,トヨタ自動車株式会社,,FAILED
0,2019-06-21 15:00:00,有価証券報告書－第115期(平成30年4月1日－平成31年3月31日),E02144,トヨタ自動車株式会社,,FAILED
0,2018-06-25 15:00:00,有価証券報告書－第114期(平成29年4月1日－平成30年3月31日),E02144,トヨタ自動車株式会社,,FAILED
0,2017-06-23 16:04:00,有価証券報告書－第113期(平成28年4月1日－平成29年3月31日),E02144,トヨタ自動車株式会社,,FAILED


In [ ]:
ylinks = ys.get_yuho_links()
len(ylinks)
for ylink in ylinks:
    ys.click_yuho_link(ylink)
    my.click_holdings_section()
    my.save_content('kaishajokyo')
    my.close_all_but_first_window()    

In [203]:
del ys

need to implement 
- click yuho link 15 min
- get_yuho_links 15 min

started at 11:48

In [ ]:
ys = YuhoScraper()
%%time
my = ostRecentYuhoScraper()
my.start_driver()
failed_codes = []
codes = ['7203', '7201']
for code in codes:
    try:
        my.get_search_page_from_code(code)
        my.click_yuho_link()
        my.save_content('hyoushi')
        my.click_holdings_section()
        my.save_content('kaishajokyo')
        my.close_all_but_first_window()
    except (TimeoutException, ElementNotInteractableException):
        print("{code} failed.".format(code=code))
        failed_codes.append(code)

In [196]:
rawurl

'https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.verb=W1E63021CXP002002DSPSch&uji.bean=ee.bean.parent.EECommonSearchBean&PID=W1E63021&TID=W1E63021&SESSIONKEY=1626160755834&lgKbn=2&pkbn=0&skbn=1&dskb=&askb=&dflg=0&iflg=0&preId=1&sec={edinetcode}&scc=&shb=&snm=&spf1=1&spf2=1&iec=&icc=&inm=&spf3=1&fdc=&fnm=&spf4=1&spf5=2&otd=120&cal=1&era=R&yer=&mon=&psr=1&pfs=5&row=100&idx=0&str=&kbn=1&flg=&syoruiKanriNo='

In [195]:
for i, row in uni.reset_index(drop=True).iterrows():
    print(i)
    if i > 10:
        break
    xlnk = rawurl
    print(row)
    driver = webdriver.Chrome(fn, chrome_options=options, 
                          desired_capabilities=DesiredCapabilities.CHROME)
    rdf = extract_holders_by_edinetcode(driver, row['ＥＤＩＮＥＴコード'],
                                        row['ticker'], xlnk)
    rdf = prepare_record_dataframe(rdf)
    driver.quit()

0
日付                                    20210531
銘柄名                                       トヨタ自
ticker                                    7203
業種                                       輸送用機器
TOPIXに占める個別銘柄のウェイト                    0.034987
ニューインデックス区分                       TOPIX Core30
調整係数対象銘柄                                   NaN
ＥＤＩＮＥＴコード                               E02144
提出者種別                                  内国法人・組合
上場区分                                        上場
連結の有無                                        有
資本金                                     635401
決算日                                      3月31日
提出者名                                トヨタ自動車株式会社
提出者名（英字）              TOYOTA MOTOR CORPORATION
提出者名（ヨミ）                      トヨタジドウシャカブシキカイシャ
所在地                                 豊田市トヨタ町１番地
提出者業種                                    輸送用機器
証券コード                                    72030
提出者法人番号                          1180301018771
wgt                                   0.034987
Name: 0, dt

C:\Users\Pawlovicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.verb=W1E63021CXP002002DSPSch&uji.bean=ee.bean.parent.EECommonSearchBean&PID=W1E63021&TID=W1E63021&SESSIONKEY=1626160755834&lgKbn=2&pkbn=0&skbn=1&dskb=&askb=&dflg=0&iflg=0&preId=1&sec=E02144&scc=&shb=&snm=&spf1=1&spf2=1&iec=&icc=&inm=&spf3=1&fdc=&fnm=&spf4=1&spf5=2&otd=120&cal=1&era=R&yer=&mon=&psr=1&pfs=5&row=100&idx=0&str=&kbn=1&flg=&syoruiKanriNo=
E02144 - 有価証券報告書－第117期(令和2年4月1日－令和3年3月31日)


NoSuchFrameException: Message: mainFrame


In [193]:
xlnk = rawurl 
ticker = '7203'
edinetcode = 'E02144'
driver = webdriver.Chrome(fn, chrome_options=options, 
                          desired_capabilities=DesiredCapabilities.CHROME)
rdf = extract_holders_by_edinetcode(driver, edinetcode, ticker, xlnk)
rdf = prepare_record_dataframe(rdf)
driver.quit()

C:\Users\Pawlovicky\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


E02144 - 有価証券報告書－第117期(令和2年4月1日－令和3年3月31日)


NoSuchFrameException: Message: mainFrame


In [ ]:
rdf

In [166]:

tdf = prepare_record_dataframe(rdf)
tdf

,提出日時,提出書類,コード,提出者／ファンド,発行／対象／子会社／臨報提出事由,PDF,XBRL,比較,備考,data_date
0,R3.06.22 09:13,有価証券報告書－第119期(令和2年4月1日－令和3年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN,2021-06-22 09:13:00
1,R2.06.23 09:01,有価証券報告書－第118期(平成31年4月1日－令和2年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN,2020-06-23 09:01:00
2,R1.06.24 09:02,有価証券報告書－第117期(平成30年4月1日－平成31年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN,2019-06-24 09:02:00
3,H30.06.25 16:13,有価証券報告書－第116期(平成29年4月1日－平成30年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN,2018-06-25 16:13:00
4,H29.06.26 15:09,有価証券報告書－第115期(平成28年4月1日－平成29年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN,2017-06-26 15:09:00


In [163]:
def wareki_asci_to_year(date_str):
    year = int(re.findall(r"(\d+)\.", date_str)[0])
    if 'H' in date_str:
        year = year + 1988  
    elif 'R' in date_str:
        year = year + 2019
    date_str = str(year) + '.' + '.'.join(date_str.split('.')[1:])
    return date_str

In [162]:
pd.to_datetime([dtstr], errors='coerce')

DatetimeIndex(['2021-06-23 09:01:00'], dtype='datetime64[ns]', freq=None)

In [148]:
tdf

,提出日時,提出書類,コード,提出者／ファンド,発行／対象／子会社／臨報提出事由,PDF,XBRL,比較,備考
0,R3.06.22 09:13,有価証券報告書－第119期(令和2年4月1日－令和3年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN
1,R2.06.23 09:01,有価証券報告書－第118期(平成31年4月1日－令和2年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN
2,R1.06.24 09:02,有価証券報告書－第117期(平成30年4月1日－平成31年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN
3,H30.06.25 16:13,有価証券報告書－第116期(平成29年4月1日－平成30年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN
4,H29.06.26 15:09,有価証券報告書－第115期(平成28年4月1日－平成29年3月31日),E06141,第一生命ホールディングス株式会社,NaN,NaN,NaN,NaN,NaN


In [147]:
tdf['提出日時']

0     R3.06.22 09:13
1     R2.06.23 09:01
2     R1.06.24 09:02
3    H30.06.25 16:13
4    H29.06.26 15:09
Name: 提出日時, dtype: object

In [ ]:
rdf.columns

In [141]:
rdf['提出日時']

KeyError: '提出日時'

In [140]:
pd.to_datetime(rdf['提出日時'])

KeyError: '提出日時'

In [ ]:
driver.quit()

In [ ]:
rdf

In [62]:
pd.to_datetime(['令和元年6月24日'])

ParserError: Unknown string format: 令和元年6月24日

In [ ]:
[extract_holders_by_code(str(x), xlnk) for x in tpx['コード'].tolist()[1814:]]

In [8]:
tpx = pd.read_excel('https://www.jpx.co.jp/markets/indices/topix/tvdivq00000030ne-att/TOPIX_weight_jp.xlsx')
tpx = tpx.loc[~tpx['コード'].isnull()]
tpx['コード'] = tpx['コード'].astype(int)
tpx.head()

,日付,銘柄名,コード,業種,TOPIXに占める個別銘柄のウェイト,ニューインデックス区分,調整係数対象銘柄
0,20180731,極洋,1301,水産・農林業,0.000061,TOPIX Small 2,NaN
1,20180731,日水,1332,水産・農林業,0.000277,TOPIX Mid400,NaN
2,20180731,マルハニチロ,1333,水産・農林業,0.000353,TOPIX Mid400,NaN
3,20180731,ホウスイ,1352,卸売業,0.000003,TOPIX Small 2,NaN
4,20180731,カネコ種苗,1376,水産・農林業,0.000025,TOPIX Small 2,○


In [ ]:
[extract_holders_by_code(str(x), xlnk) for x in tpx['コード'].tolist()[1814:]]

8750 - 有価証券報告書... failed to retrieve
8750 - 有価証券報告書... failed to retrieve
8750 - 有価証券報告書... failed to retrieve
8750 - 有価証券報告書... failed to retrieve
8750 - 有価証券報告書... failed to retrieve
8798 - 有価証券報告書－第18期(平成24年10月1日－平成25年9月30日)... failed to retrieve
8904 - 有価証券報告書－第24期(平成24年9月1日－平成25年8月31日)... failed to retrieve
8917 - 有価証券報告書－第15期(平成24年11月1日－平成25年10月31日)... failed to retrieve
8923 - 有価証券報告書－第64期(平成24年12月1日－平成25年11月30日)... failed to retrieve
8928 - 有価証券報告書－第50期(平成24年7月1日－平成25年6月30日)... failed to retrieve
8933 - 有価証券報告書－第33期(平成29年4月1日－平成30年3月31日)... failed to retrieve
8933 - 有価証券報告書－第32期(平成28年4月1日－平成29年3月31日)... failed to retrieve
8933 - 有価証券報告書－第31期(平成27年4月1日－平成28年3月31日)... failed to retrieve
8933 - 有価証券報告書－第30期(平成26年4月1日－平成27年3月31日)... failed to retrieve
8933 - 有価証券報告書－第29期(平成25年4月1日－平成26年3月31日)... failed to retrieve
8944 - 有価証券報告書－第29期(平成24年10月1日－平成25年9月30日)... failed to retrieve
9260 - 有価証券報告書－第27期(平成24年10月1日－平成25年9月30日)... failed to retrieve
9310 - 有価証券報告書－第9期(平成24年10月1日－平成25年9月30日

In [16]:
 tpx['コード'].tolist()[1814:]

[8750,
 8766,
 8769,
 8771,
 8772,
 8793,
 8795,
 8798,
 8801,
 8802,
 8803,
 8804,
 8806,
 8818,
 8830,
 8835,
 8840,
 8841,
 8842,
 8848,
 8850,
 8860,
 8864,
 8869,
 8871,
 8876,
 8877,
 8881,
 8892,
 8897,
 8904,
 8905,
 8917,
 8918,
 8919,
 8920,
 8923,
 8928,
 8933,
 8934,
 8935,
 8940,
 8944,
 8999,
 9001,
 9003,
 9005,
 9006,
 9007,
 9008,
 9009,
 9010,
 9014,
 9020,
 9021,
 9022,
 9024,
 9025,
 9031,
 9037,
 9039,
 9041,
 9042,
 9044,
 9045,
 9046,
 9048,
 9052,
 9058,
 9062,
 9064,
 9065,
 9066,
 9067,
 9068,
 9069,
 9070,
 9072,
 9074,
 9075,
 9076,
 9078,
 9081,
 9086,
 9090,
 9099,
 9101,
 9104,
 9107,
 9110,
 9115,
 9119,
 9130,
 9142,
 9143,
 9201,
 9202,
 9232,
 9260,
 9265,
 9267,
 9274,
 9301,
 9302,
 9303,
 9304,
 9305,
 9306,
 9308,
 9310,
 9312,
 9319,
 9322,
 9324,
 9351,
 9358,
 9364,
 9366,
 9368,
 9369,
 9375,
 9380,
 9381,
 9384,
 9386,
 9401,
 9404,
 9405,
 9409,
 9412,
 9413,
 9414,
 9416,
 9418,
 9419,
 9422,
 9424,
 9428,
 9432,
 9433,
 9435,
 9437,
 9438,

In [ ]:
extract_holders_by_code(str(1414), xlnk)

In [ ]:
''.join('2018年 6月28日'.split())
